In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")
import re
import nltk
import dask.array as da

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(encoding='utf-8')

## Векторизация текста на основе модели "мешка слов"

Векторизация текста поддерживается библиотекой [scikit-learn](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

### Использование класса [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [46]:
data = pd.read_csv('data/tags.csv')

In [47]:
data.head()

,name,tags
0,Благословение небожителей. Том 2,все в эт мир имет счет и удач и невезен бед пр...
1,Благословение небожителей. Том 1,в незапамятн врем си лянут быт наследн принц г...
2,Лето в пионерском галстуке,юр возвраща в пионерск лагер сво юност спуст д...
3,Электрошок. Внезапно,нов книг от автор культов цикл древн наслед и ...
4,Билли Саммерс,нов увлекательн рома от автор культов бестселл...


Подсчитывает количество слов словаря, входящих в данный текст.

In [48]:
cv.fit(data['tags'])
corpusVocab = cv.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 40132


In [49]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

мир=18083
имет=11807
счет=33625
удач=35436
невезен=19819
бед=1917
преследова=26493
си=31002
лян=16811


In [9]:
vectors = cv.fit_transform(data['tags'].values.astype('U'))

In [10]:
vectors

<9070x46222 sparse matrix of type '<class 'numpy.int64'>'
	with 602058 stored elements in Compressed Sparse Row format>

In [11]:
vectors.shape

(9070, 46222)

In [12]:
vectors.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
# Непустые значения нулевой строки
[i for i in vectors.todense()[0].getA1() if i>0][0:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [14]:
cv.get_feature_names()[23567:23587]

C:\Users\kenia\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['непродолжительный',
 'непрожитый',
 'непроизвольно',
 'непроизвольный',
 'непройти',
 'непролазный',
 'непроницаемый',
 'непростительный',
 'непросто',
 'непростой',
 'непротиворечивый',
 'непроходимый',
 'непроходить',
 'непрочность',
 'непрочный',
 'непрочь',
 'непрошенный',
 'непрошеный',
 'непрощённый',
 'нептун']

### CountVectorizer with spacy

In [5]:
data = pd.read_csv('data/spacylem.csv')
data.head()

,name,tags
0,Благословение небожителей. Том 2,ве мир имет счет удач невезен бед преследова с...
1,Благословение небожителей. Том 1,незапамятн врем се лян наследн принц государст...
2,Лето в пионерском галстуке,юр возвраща пионерск лагер сво юност спуст два...
3,Электрошок. Внезапно,нов книг автор культов цикл древн наслед кажд ...
4,Билли Саммерс,нов увлекательн рома автор культов бестселлер ...


In [6]:
cv.fit(data['tags'])
corpusVocab = cv.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 42206


In [7]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

мир=18862
имет=12423
счет=35449
удач=37344
невезен=20692
бед=1969
преследова=27851
се=32295
лян=17557


In [8]:
vectors = cv.fit_transform(data['tags'].values.astype('U'))
# Непустые значения нулевой строки
[i for i in vectors.todense()[0].getA1() if i>0][0:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [9]:
cv.get_feature_names()[29443:29463]

C:\Users\kenia\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['пышет',
 'пышк',
 'пышн',
 'пышногруд',
 'пышност',
 'пьедеста',
 'пьедр',
 'пьемонт',
 'пьер',
 'пьес',
 'пьеск',
 'пьет',
 'пьетр',
 'пьех',
 'пьюджет',
 'пьюз',
 'пьюзон',
 'пьюзоскандинавск',
 'пьян',
 'пьянеет']

### Cosine_similarity

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
%%time
similarity = cosine_similarity(vectors)

CPU times: total: 3.84 s
Wall time: 7.42 s


In [12]:
similarity

array([[1.        , 0.2699865 , 0.11253787, ..., 0.02007239, 0.12501206,
        0.        ],
       [0.2699865 , 1.        , 0.12909055, ..., 0.01465211, 0.1520904 ,
        0.05695718],
       [0.11253787, 0.12909055, 1.        , ..., 0.        , 0.1467153 ,
        0.04578685],
       ...,
       [0.02007239, 0.01465211, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.12501206, 0.1520904 , 0.1467153 , ..., 0.        , 1.        ,
        0.11867817],
       [0.        , 0.05695718, 0.04578685, ..., 0.        , 0.11867817,
        1.        ]])

In [13]:
similarity[0]

array([1.        , 0.2699865 , 0.11253787, ..., 0.02007239, 0.12501206,
       0.        ])

In [14]:
type(similarity)

numpy.ndarray

In [15]:
list(enumerate(similarity[0]))[:10]

[(0, 1.0000000000000013),
 (1, 0.26998650101241567),
 (2, 0.11253786588843415),
 (3, 0.03901371573204352),
 (4, 0.05579717762694858),
 (5, 0.02136869215853441),
 (6, 0.10148754557830483),
 (7, 0.09363291775690445),
 (8, 0.02758686295341227),
 (9, 0.10655033689112361)]

In [17]:
sorted(similarity[0],reverse=True)[:10]

[1.0000000000000013,
 0.26998650101241567,
 0.22635466595164022,
 0.20516372588964538,
 0.20307285509118606,
 0.19726137691880566,
 0.19726137691880563,
 0.19715227490949905,
 0.19633468379247637,
 0.1932566996956512]

In [18]:
def recommend(rusbook):
    rusbook_index = data[data['name'] == rusbook].index[0]
    distances = similarity[rusbook_index]
    rusbook_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in rusbook_list:
       print(data.name[data.iloc[i[0]].name])

In [19]:
recommend('Благословение небожителей. Том 2 ')

Благословение небожителей. Том 1 
Золотое сердце Вавилона 
Забыть нельзя вспомнить 
В Калифорнии морозов не бывает 
Дивергент 
Долорес Клейборн 
Король говорит! История о преодолении, о долге и чести, о лидерстве, об иерархии и о настоящей дружбе 
Против ветра, мимо облаков 
В академию за хозяином Драконьего края 
Через год в это же время 


In [20]:
recommend('Тень и кость ')

Штурм и буря 
Серафина 
Нежно 
Шёпот магии 
Счастье рядом 
Эхо между нами 
Братья Карилло. Обретая надежду 
Песня черного ангела 
Химеры картинной галереи 
Зов темной крови 


In [25]:
recommend('Шестерка воронов ')

Тень и кость 
Штурм и буря 
Мой Рагнарек 
Крах и восход 
Изоморф. Вор 
Король шрамов 
Когда восходит тень 
Продажное королевство 
Тени сгущаются 
Украденное время 


In [33]:
recommend('Герой нашего времени ')

Темница тихого ангела. Желать невозможного 
Обломов 
Исповедь. О жизни 
Любовь без права на ошибку 
Вот я 
Война и мир. I-II 
Война и мир. III-IV 
Пока еще здесь 
Перед восходом солнца 
Портрет Дориана Грея 


### Выгружаем

In [37]:
import pickle

In [38]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [41]:
pickle.dump(data.to_dict(),open('books_dict.pkl','wb'))